In [15]:
import pandas as pd

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium_stealth import stealth
import random
import time
import os
import unidecode

Scrap lien drive

In [16]:
def scrap_lien_drive(driver, list_lien_drive=[]): 
    
    driver.get("https://www.auchan.fr/nos-magasins?types=DRIVE")
    time.sleep(1)
    button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[11]/div[2]/div/div/div[3]/button')))
    button.click()
    time.sleep(1)
    
    footer = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="cms-slot-footerSlot"]'))
        )
    
    driver.execute_script("arguments[0].scrollIntoView(true);", footer) 

    department_tag = driver.find_elements(by = By.CLASS_NAME, value= 'storeListDepartment')
    
    for department in department_tag:

        lien_drives = department.find_elements(by = By.TAG_NAME, value= 'a')
        for lien in lien_drives:
            lien_drive = lien.get_attribute("href")
            list_lien_drive.append(lien_drive)

    return(list_lien_drive)

Scrap attribut drive

In [17]:
def scrap_drive(driver,list_lien_drive,list_drive=[]):
    for lien in list_lien_drive:

            driver.get(lien)
            
            nom_drive = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'site-breadcrumb__title--ellipsis')))
            
            adress_drive = driver.find_element(by = By.CLASS_NAME, value='store-info__address')

            coord = driver.find_element(by = By.XPATH, value='/html/body/div[3]/div[2]/div[2]/div[2]/div')
            #print(coord)
            latitude = coord.get_attribute("data-lat")
            longitude = coord.get_attribute("data-lng")

            

            dict_drive ={
                
                "nom_drive": nom_drive.text,
                'adresse': adress_drive.text,
                "latitude": latitude,
                "longitude": longitude,      
                "lien_drive": lien,

            }
            #print(dict_drive)
            list_drive.append(dict_drive)
            
            
     
    df_drive = pd.DataFrame(list_drive)
    df_drive.to_csv("list_drives_auchan.csv")
    
    return(df_drive)
        

find last index of a specific character in string 


In [18]:
def find_last_index(string, char, indices = []):
    for i in range(len(string)):
        if string[i] == char:
            indices.append(i)

    last_indice = int(indices[-1])
    return(last_indice)

find second index of a specific character in string 

In [19]:
def find_sec_index(string, char, indices = []):
    for i in range(len(string)):
        if string[i] == char:
            indices.append(i)

    last_indice = int(indices[1])
    return(last_indice)

DRIVE NETTOYé

In [20]:
def process_drive(df,list_drives = []):
    for i,adresse in enumerate(df["adresse"]):

        adresse = adresse.replace("\n",",")    
        virgule = find_last_index(string = adresse, char=',')
        department = adresse[virgule+2:virgule+4]
        city = adresse[virgule+2:]

        reservedCharacter = ["!","*","'","(",")",";",":","@","&","=","+","$",",","/","?","%","#","[","]"," "]
        nom_driveUrl = df.iloc[i]['nom_drive']
        for character in reservedCharacter:
            nom_driveUrl = nom_driveUrl.replace(character,"_")
        nom_driveUrl = unidecode.unidecode(nom_driveUrl)
        dict_drive ={
            "supermarket":"Auchan",
            "department": department,
            "nom_drive": df.iloc[i]['nom_drive'],
            "nom_driveUrl" : nom_driveUrl,
            'city': city,
            "adresse": adresse,
            "latitude": df.iloc[i]['latitude'],
            "longitude": df.iloc[i]['longitude'],          
            "lien_drive" : df.iloc[i]['lien_drive']

        }
        list_drives.append(dict_drive)
        print(dict_drive)
    df_drives = pd.DataFrame(list_drives)
    dropSuper = ["Souppes-sur-loing"]
    df_drives = df_drives[~df_drives['nom_drive'].str.contains('|'.join(dropSuper))]
    
    df_drives.to_csv("list_drives_auchan.csv")

In [21]:
#set up driver
# create a new Service instance and specify path to Chromedriver executable
service = ChromeService(executable_path=ChromeDriverManager().install())


# Step 2: Change browser properties
# create a ChromeOptions object
options = webdriver.ChromeOptions()

#run in headless mode
#options.add_argument("--headless")

# disable the AutomationControlled feature of Blink rendering engine
options.add_argument('--disable-blink-features=AutomationControlled')
 
# disable pop-up blocking
options.add_argument('--disable-popup-blocking')
 
# start the browser window in maximized mode
options.add_argument('--start-maximized')
 
# disable extensions
options.add_argument('--disable-extensions')
 
# disable sandbox mode
options.add_argument('--no-sandbox')
 
# disable shared memory usage
options.add_argument('--disable-dev-shm-usage')


# Set navigator.webdriver to undefined
# create a driver instance
driver = webdriver.Chrome(service=service, options=options)

# Change the property value of the navigator for webdriver to undefined
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")



# Step 3: Rotate user agents 
user_agents = [
    # Add your list of user agents here
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]

# select random user agent
user_agent = random.choice(user_agents)

# pass in selected user agent as an argument
options.add_argument(f'user-agent={user_agent}')

# set user agent using execute_cpd_cmd
driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})


# Step 4: Scrape using Stealth
#enable stealth mode
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,
        )
    
list_lien_drive = scrap_lien_drive(driver)
    
df=scrap_drive(driver,list_lien_drive)
    
process_drive(df)
    

{'supermarket': 'Auchan', 'department': '02', 'nom_drive': 'Auchan Drive St-quentin', 'nom_driveUrl': 'Auchan_Drive_St-quentin', 'city': '02100 Fayet', 'adresse': 'Cc Quentin De La Tour , 02100 Fayet', 'latitude': '49.858073', 'longitude': '3.256202', 'lien_drive': 'https://www.auchan.fr/magasins/drive/st-quentin/s-985'}
{'supermarket': 'Auchan', 'department': '02', 'nom_drive': 'Auchan Drive Hirson', 'nom_driveUrl': 'Auchan_Drive_Hirson', 'city': '02500 Hirson', 'adresse': 'Avenue De Verdun , 02500 Hirson', 'latitude': '49.91261', 'longitude': '4.09081', 'lien_drive': 'https://www.auchan.fr/magasins/drive/hirson/s-990'}
{'supermarket': 'Auchan', 'department': '02', 'nom_drive': 'Auchan Drive Viry Noureuil', 'nom_driveUrl': 'Auchan_Drive_Viry_Noureuil', 'city': '02300 Viry-noureuil', 'adresse': 'Route Nationale 32 , 02300 Viry-noureuil', 'latitude': '49.64375', 'longitude': '3.25926', 'lien_drive': 'https://www.auchan.fr/magasins/drive/viry-noureuil/s-551'}
{'supermarket': 'Auchan', 'd